In [26]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from glob import glob
from dateutil.relativedelta import relativedelta,TH
import warnings
warnings.filterwarnings("ignore")

In [27]:
### Total one month data in a single File.  I create a one function and this function is convert single file to Date wise data file.
    

def Data_preparation(Data):
    # Read Data_set 
    data = pd.read_csv(Data)
    # Seprate Banknifty Future Price in dataset
    
    df =data[data['Ticker'] == 'BANKNIFTY-FUT'].reset_index(drop = 'Frist')
    # create New columns 'Instrument'
    
    df['Instrument'] = 'FUTIDX'
    df['Strike_price'] = '00'
    
    # Create Symbol
    df['Symbol'] = df['Ticker'].apply(lambda x: x.split('-')[0])
    
    # Create Timestamp
    df['Timestamp']=df['Date/Time'].apply(lambda x: x.split(' ')[0])
    
    # Create Nifty Fut dataframe and Timestamp
    df1 =data[data['Ticker'] == 'NIFTY-FUT'].reset_index(drop = True)
    df1['Instrument'] ='FUTIDX'
    df1['Strike_price'] = "00"
    df1['Symbol']=df1['Ticker'].apply(lambda x: x.split('-')[0])
    df1['Timestamp'] =df1['Date/Time'].apply(lambda x: x.split(' ')[0])
    
    # Create new dataframe and store option data and drop not required Data
    no_data = ['BANKNIFTY' , 'BANKNIFTY-FUT' , 'NIFTY' , 'NIFTY-FUT' , 'FINNIFTY-FUT' , 'INDIAVIX']
    df2 = data[~data['Ticker'].isin(no_data)].reset_index(drop = 'Frist')
    df2['Instrument'] = 'OPTIDX'
    df3 =df2['Ticker'].apply(lambda x: x.split('Y')[-1].split('CE')[0])
    df2['Strike_price'] = df3.apply(lambda x: x.split('PE')[0])
    df2['Option_Type'] = df2['Ticker'].apply(lambda x: x.split('Y')[-1].split('0')[-1])
    df2['Symbol'] = df2['Ticker'].apply(lambda x: x.split('Y')[0])
    df2['Symbol'] = df2['Symbol'].replace({'BANKNIFT' : 'BANKNIFTY' , 'NIFT' : 'NIFTY'})
    df2['Timestamp']=df2['Date/Time'].apply(lambda x: x.split(" ")[0])
    
    # marge all data frame in sigle dataframe
    new_data = pd.concat([df,df1,df2])
    new_data['Option_Type']=new_data['Option_Type'].fillna('XX')
    
    # drop Not Required columns
    new_data.drop(['Ticker' , 'Volume' , 'Open Interest'], axis=1 , inplace = True)
    new_data['Strike_price'] = new_data['Strike_price'].astype('int')
    for i in new_data['Timestamp'].unique():
        final = new_data[new_data['Timestamp'] == i]
        # Store data in date wise file
        final.to_csv(f'Option_data_{i}' , index=False)

In [28]:
Data_preparation('AUGUST-2022-MONTHLY-Expiry-data _Vege.csv')